---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

--- 2025

# Webcam + Transformations simples avec OpenCV

Dans ce notebook, nous allons utiliser la **webcam** pour afficher en temps réel :

- l'image originale de la webcam,
- une version **redimensionnée** (Resize),
- une version **translatée** (Translation),
- une version **rotatée** (Rotation),
- une version **croppée** (Crop),
- une version **zoomée** (Zoom = crop + agrandissement),
- une version **flippée** (Flip = miroir).

Chaque transformation sera affichée dans une **fenêtre différent**e, en direct.

> ⚠ Ce notebook est conçu pour être exécuté en local (par exemple avec VS Code ou Jupyter sur ton ordinateur) avec une webcam accessible par OpenCV. Sur Colab, les fenêtres `cv2.imshow` ne fonctionneront pas de cette façon.

## 1. Installation de OpenCV (si nécessaire)
À exécuter uniquement si `opencv-python` n'est pas déjà installé dans ton environnement.

In [ ]:
%%bash
# À lancer une fois si besoin
pip install opencv-python

## 2. Paramètres des transformations et capture webcam

Tu peux modifier les paramètres ci-dessous (taille de resize, translation, angle de rotation, zone de crop, facteur de zoom, type de flip) directement dans le code. Ensuite, exécute la cellule pour lancer la capture webcam.

- Appuie sur la touche **`q`** dans une des fenêtres pour arrêter la capture et fermer toutes les fenêtres.


In [ ]:
import cv2
import numpy as np

# ----------------------------
# Paramètres des transformations
# ----------------------------
# 1) Resize
resize_width = 320   # largeur souhaitée
resize_height = 240  # hauteur souhaitée

# 2) Translation (en pixels)
tx, ty = 50, 30  # déplacement vers la droite (tx>0) et vers le bas (ty>0)

# 3) Rotation
angle = 30   # en degrés (sens anti-horaire)
scale = 1.0  # facteur d'échelle (1.0 = pas de zoom supplémentaire)

# 4) Crop (on définira une zone centrale relative à la taille du frame)
#    Les valeurs sont des fractions de la hauteur/largeur
crop_y1_ratio, crop_y2_ratio = 0.25, 0.75
crop_x1_ratio, crop_x2_ratio = 0.25, 0.75

# 5) Zoom : on va prendre le crop et le redimensionner à la taille originale
#    Rien à régler ici directement, il dépend du crop.

# 6) Flip
# flip_code = 1  -> flip horizontal (gauche/droite)
# flip_code = 0  -> flip vertical (haut/bas)
# flip_code = -1 -> flip horizontal + vertical
flip_code = 1

# ----------------------------
# Capture webcam
# ----------------------------
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Impossible d'ouvrir la webcam (index 0)")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erreur : impossible de lire un frame depuis la webcam.")
        break

    # Optionnel : effet miroir pour que la webcam se comporte comme un miroir
    frame = cv2.flip(frame, 1)

    # Dimensions de l'image
    h, w = frame.shape[:2]

    # 1) Resize
    frame_resize = cv2.resize(frame, (resize_width, resize_height))

    # 2) Translation
    M_translation = np.float32([[1, 0, tx],
                                [0, 1, ty]])
    frame_translation = cv2.warpAffine(frame, M_translation, (w, h))

    # 3) Rotation autour du centre
    center = (w // 2, h // 2)
    M_rotation = cv2.getRotationMatrix2D(center, angle, scale)
    frame_rotation = cv2.warpAffine(frame, M_rotation, (w, h))

    # 4) Crop (zone centrale)
    y1 = int(crop_y1_ratio * h)
    y2 = int(crop_y2_ratio * h)
    x1 = int(crop_x1_ratio * w)
    x2 = int(crop_x2_ratio * w)

    # On s'assure que les indices sont valides
    y1 = max(0, min(y1, h-1))
    y2 = max(1, min(y2, h))
    x1 = max(0, min(x1, w-1))
    x2 = max(1, min(x2, w))

    frame_crop = frame[y1:y2, x1:x2]

    # 5) Zoom (on redimensionne le crop à la taille originale du frame)
    if frame_crop.size != 0:
        frame_zoom = cv2.resize(frame_crop, (w, h))
    else:
        frame_zoom = frame.copy()

    # 6) Flip
    frame_flip = cv2.flip(frame, flip_code)

    # ----------------------------
    # Affichage dans 7 fenêtres différentes
    # ----------------------------
    cv2.imshow('Webcam - Original', frame)
    cv2.imshow('Webcam - Resize', frame_resize)
    cv2.imshow('Webcam - Translation', frame_translation)
    cv2.imshow('Webcam - Rotation', frame_rotation)
    cv2.imshow('Webcam - Crop', frame_crop)
    cv2.imshow('Webcam - Zoom (crop + resize)', frame_zoom)
    cv2.imshow('Webcam - Flip', frame_flip)

    # Touche 'q' pour quitter
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 3. Pistes de questions / prolongements

- Que se passe-t-il si on augmente beaucoup la translation ?
- Que remarquez-vous lorsque l'on change l'angle de rotation ?
- Comment évolue la "qualité" de l'image lorsque l'on zoom fortement (crop très petit) ?
- À votre avis, comment ces transformations peuvent-elles être utiles en **intelligence artificielle** (par exemple pour entraîner un modèle de vision par ordinateur) ?


---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

--- 2025